# 1 - Data Discovery & Exploratory Profiling  
## Dataset: User Top Artists / Albums / Tracks  
### Fase 0 — Landing Zone Analysis  
**Autor:** Guilherme Brevilato  
**Objetivo:**  
Realizar inspeção exploratória dos arquivos RAW em CSV presentes na Landing Zone, identificando:  
- estrutura  
- colunas  
- qualidade  
- possíveis inconsistências (schema drift / data anomalies)  
- inferência inicial do domínio  
- riscos técnicos para ingestão Bronze  


In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import os

spark = (
    SparkSession.builder
    .appName("discovery_profiling")
    .config("spark.sql.shuffle.partitions", "50")   # menor número para exploração
    .config("spark.sql.adaptive.enabled", "true")
    .getOrCreate()
)

spark


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/12/01 11:46:20 WARN Utils: Your hostname, MacBook-Pro-de-Guilherme.local, resolves to a loopback address: 127.0.0.1; using 192.168.15.135 instead (on interface en0)
25/12/01 11:46:20 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/01 11:46:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## 2 - Paths

### Landing zone

In [2]:
RAW = "/Volumes/external/Dados/dataset_2/archive/rawcsv"

## 3 - File-Level Inspection

In [15]:
files = [f for f in os.listdir(RAW) if f.endswith(".csv")]
files

['user_top_albums.csv',
 '._user_top_albums.csv',
 'user_top_artists.csv',
 '._user_top_artists.csv',
 'user_top_tracks.csv',
 '._user_top_tracks.csv',
 'users.csv',
 '._users.csv']

## 4 - Head-Level Sampling

In [14]:
for file in files:
    print(f"\n=== HEAD of {file} ===\n")
    !head -n 5 "$RAW/$file"


=== HEAD of user_top_albums.csv ===

user_id,rank,album_name,artist_name,playcount,mbid
1,1,Visions,Grimes,243,0136a543-33fa-4b80-a8ca-4a6094db3b71
1,2,Only Built 4 Cuban Linx,Raekwon,240,c49af812-cb43-45ca-81e6-b81b7c287624
1,3,Tales From the Pantry,Lederhosen Lucil,204,1f195540-653f-4db4-bb74-e7d215de9bae
1,4,Crystal Castles,Crystal Castles,200,101f993f-d5e1-3f14-b40b-71850f1f435a

=== HEAD of ._user_top_albums.csv ===

    Mac OS X            	   2  �     �                                    ATTR      �   �   9                  �   9  com.apple.quarantine 0281;6925eeb1;Chrome;69766E16-C1F8-4FEA-B748-2CF6B0E01F6E                                                                                                                                                                                                                                                                                                                                                                            

## 5 - Exploratory Load

In [5]:
dfAr_raw = (
    spark.read
        .option("header", True)
        .option("inferSchema", True) 
        .csv(f"{RAW}/user_top_artists.csv")
)

dfAl_raw = (
    spark.read
        .option("header", True)
        .option("inferSchema", True)
        .csv(f"{RAW}/user_top_albums.csv")
)

dfTr_raw = (
    spark.read
        .option("header", True)
        .option("inferSchema", True)
        .csv(f"{RAW}/user_top_tracks.csv")
)


## 6 - Contract Hypothesis

In [6]:
dfAr_raw.printSchema()
dfAl_raw.printSchema()
dfTr_raw.printSchema()


root
 |-- user_id: integer (nullable = true)
 |-- rank: integer (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- playcount: string (nullable = true)
 |-- mbid: string (nullable = true)

root
 |-- user_id: integer (nullable = true)
 |-- rank: integer (nullable = true)
 |-- album_name: string (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- playcount: string (nullable = true)
 |-- mbid: string (nullable = true)

root
 |-- user_id: integer (nullable = true)
 |-- rank: integer (nullable = true)
 |-- track_name: string (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- playcount: string (nullable = true)
 |-- mbid: string (nullable = true)



## 7 - quality patterns

### Row-Level Profiling

In [7]:
dfAr_raw.limit(10).show(truncate=False)
dfAl_raw.limit(10).show(truncate=False)
dfTr_raw.limit(10).show(truncate=False)


+-------+----+----------------+---------+------------------------------------+
|user_id|rank|artist_name     |playcount|mbid                                |
+-------+----+----------------+---------+------------------------------------+
|1      |1   |Crystal Castles |1034     |b1570544-93ab-4b2b-8398-131735394202|
|1      |2   |Radiohead       |972      |a74b1b7f-71a5-4011-9441-d0b5e4122711|
|1      |3   |Ladytron        |831      |b45335d1-5219-4262-a44d-936aa36eeaed|
|1      |4   |Ghostface Killah|801      |3b39abeb-0064-4eed-9ddd-ee47a45c54cb|
|1      |5   |UNKLE           |722      |6648391e-7890-4f6c-b939-976f215195d3|
|1      |6   |DJ Shadow       |711      |efa2c11a-1a35-4b60-bc1b-66d37de88511|
|1      |7   |Buck 65         |700      |8d18b680-368c-4649-a5e3-85e0c2dd6fc2|
|1      |8   |AFI             |671      |NULL                                |
|1      |9   |Raekwon         |593      |4e954b02-fae2-4bd7-9547-e055a6ac0527|
|1      |10  |Misfits         |565      |936addc3-91

### Volume Assessment

In [8]:
print("Artists:", dfAr_raw.count())
print("Albums :", dfAl_raw.count())
print("Tracks :", dfTr_raw.count())


Artists: 23822550
Albums : 23822550
Tracks : 23822550


### Cardinalidade e integridade

In [9]:
print("Artists:", dfAr_raw.select('artist_name').distinct().count())
print("Album:", dfAl_raw.select('artist_name').distinct().count())
print("Tracks:", dfTr_raw.select('artist_name').distinct().count())

Artists: 531590


Album: 482468


[Stage 30:===================================================>    (12 + 1) / 13]

Tracks: 537485


### Column-Level Profiling

In [10]:
for col in dfAr_raw.columns:
    print(col, " → ", dfAr_raw.select(col).distinct().count(), "distinct values")


user_id  →  476451 distinct values


rank  →  50 distinct values


artist_name  →  531590 distinct values


playcount  →  43693 distinct values


[Stage 60:>                                                       (0 + 12) / 12]

mbid  →  180181 distinct values


## 8 - DQ Checkpoints

### Campos Nulos

In [11]:
from pyspark.sql import functions as F

(
    dfAr_raw
      .select([
          F.sum(F.col(c).isNull().cast("int")).alias(f"{c}_nulls")
          for c in dfAr_raw.columns
      ])
).show(truncate=False)


[Stage 66:>                                                       (0 + 12) / 12]

+-------------+----------+-----------------+---------------+----------+
|user_id_nulls|rank_nulls|artist_name_nulls|playcount_nulls|mbid_nulls|
+-------------+----------+-----------------+---------------+----------+
|0            |0         |0                |0              |3495236   |
+-------------+----------+-----------------+---------------+----------+



### Schema drift detection

In [12]:
dfAr_raw.agg(
    *[
        F.countDistinct(F.col(c).cast("string")).alias(c)
        for c in dfAr_raw.columns
    ]
).show()


[Stage 69:======================================>                  (8 + 4) / 12]

+-------+----+-----------+---------+------+
|user_id|rank|artist_name|playcount|  mbid|
+-------+----+-----------+---------+------+
| 476451|  50|     531590|    43693|180180|
+-------+----+-----------+---------+------+



## Executive Summary – Fase 0: Discovery & Profiling

A análise exploratória inicial dos datasets “top_artists”, “top_albums” e “top_tracks”
permitiu compreender a estrutura, o comportamento e os riscos dos arquivos CSV
presentes na Landing Zone. Os datasets apresentam volume elevado (≈ 23,8M linhas por arquivo)
e características que impactam diretamente a modelagem e ingestão Bronze.

A partir do profiling, foi possível identificar:
- inconsistências estruturais (tipos divergentes entre colunas equivalentes),
- campos semântico-chaves altamente incompletos (ex.: MBID),
- variações de nomenclatura que exigem normalização,
- e artefatos gerados pelo macOS (arquivos iniciados com “._”).

Estas descobertas subsidiam a definição do schema explícito da Bronze e
das regras de qualidade necessárias para garantir ingestão confiável.


## Technical Conclusions & Architectural Decisions

### 1. Schema Inferido vs. Schema Oficial
O uso de `inferSchema=True` foi adequado para a fase exploratória
e permitiu observar que `playcount`, embora numérico, foi inferido como string
em todos os datasets.  
**Decisão:** converter explicitamente para `LongType` na Bronze.

### 2. Business Key para Artistas
O campo `mbid` apresenta taxa muito alta de null (≈ 3,4 milhões), tornando-o
inadequado como Business Key.  
**Decisão:** utilizar `artist_name` normalizado como chave técnica na Silver.

### 3. Normalização Necessária
O profiling revelou variação de caixa e possíveis espaços invisíveis em `artist_name`,
exigindo padronização.  
**Decisão:** aplicar `lower(trim())` obrigatoriamente na Silver.

### 4. Volume de Dados
Cada dataset possui ≈ 23,8M de linhas.  
Isso reforça a necessidade de:
- schema explícito,
- uso de Delta Lake na Bronze,
- controle de shuffle partitions,
- leitura incremental futura (se aplicável).

### 5. Arquivos Apple Metadata
Foram detectados arquivos iniciados com "._", típicos do macOS/APFS.
**Decisão:** excluir esses arquivos na Bronze usando filtro por padrão de nome.

### 6. Campos inutilizados
Campos como `url` não agregam à dimensão, mas podem ser mantidos no Bronze
por fidelidade ao raw.  
**Decisão:** manter no Bronze, avaliar uso na Silver.

###


## Technical Conclusions & Architectural Decisions

### 1. Schema Inferido vs. Schema Oficial
O uso de `inferSchema=True` foi adequado para a fase exploratória
e permitiu observar que `playcount`, embora numérico, foi inferido como string
em todos os datasets.  
**Decisão:** converter explicitamente para `LongType` na Bronze.

### 2. Business Key para Artistas
O campo `mbid` apresenta taxa muito alta de null (≈ 3,4 milhões), tornando-o
inadequado como Business Key.  
**Decisão:** utilizar `artist_name` normalizado como chave técnica na Silver.

### 3. Normalização Necessária
O profiling revelou variação de caixa e possíveis espaços invisíveis em `artist_name`,
exigindo padronização.  
**Decisão:** aplicar `lower(trim())` obrigatoriamente na Silver.

### 4. Volume de Dados
Cada dataset possui ≈ 23,8M de linhas.  
Isso reforça a necessidade de:
- schema explícito,
- uso de Delta Lake na Bronze,
- controle de shuffle partitions,
- leitura incremental futura (se aplicável).

### 5. Arquivos Apple Metadata
Foram detectados arquivos iniciados com "._", típicos do macOS/APFS.
**Decisão:** excluir esses arquivos na Bronze usando filtro por padrão de nome.

### 6. Campos inutilizados
Campos como `url` não agregam à dimensão, mas podem ser mantidos no Bronze
por fidelidade ao raw.  
**Decisão:** manter no Bronze, avaliar uso na Silver.

###


##  Risk Register — Data & Ingestion Risks (Fase 0)

###  Risco 1 — MBID altamente incompleto
- O campo `mbid` apresenta grande volume de valores nulos.
- Impacto: inviabiliza o uso do MBID como Business Key.
- Mitigação: utilizar `artist_name_key` (normalizado) como NK na Silver.
- Fallback: manter MBID apenas como atributo descritivo no Bronze.

---

###  Risco 2 — playcount inferido como STRING
- Tipo inesperado: deveria ser numérico.
- Impacto: risco de schema drift e falha em agregações futuras.
- Mitigação: definir manualmente `LongType` no schema da Bronze.

---

###  Risco 3 — Variações e ruídos em artist_name
- Variação de caixa (“Crystal Castles”, “crystal castles”, “CRYSTAL CASTLES”).
- Possível presença de múltiplos espaços.
- Impacto: duplicações falsas em dimensões.
- Mitigação: normalização obrigatória (`lower(trim())`) na Silver.

---

###  Risco 4 — Arquivos APFS/Spotlight (._*)
- Arquivos adicionais criados pelo macOS foram detectados.
- Impacto: podem quebrar a ingestão ao tentar ler .csv inválido.
- Mitigação: filtrar arquivos com prefixo “._” na Bronze.

---

###  Risco 5 — Volume elevado (> 20M registros por arquivo)
- Impacto: maior custo de inferência, shuffle e memória.
- Mitigação:
  - schema explícito na Bronze,
  - compressão Delta,
  - controle de partições
  

---

###  Risco 6 — Possível Schema Drift entre arquivos
- Mesmo dataset pode apresentar colunas com tipos diferentes entre arquivos.
- Mitigação: schema manual + `badRecordsPath`.

---

###  Risco 7 — Colunas com baixa utilidade ou redundantes
- Exemplos: `url`.
- Impacto: aumenta largura do registro sem benefício analítico.
- Mitigação: manter no Bronze (raw fidelity), avaliar descarte na Silver/Gold.



##  Principais Achados (Summary of Findings)

- Estrutura CSV consistente, com header.
- Alto volume por arquivo (~23.8M registros).
- artist_name apresenta drift de caixa → normalização necessária.
- MBID extremamente incompleto → não é chave confiável.
- playcount veio como texto → precisa conversão para LongType.
- Arquivos-metadata do macOS detectados (._*) → filtrar na Bronze.
- track_name e album_name possuem variações de caracteres especiais.
- Campos irrelevantes podem ser mantidos apenas no Bronze.
- Nenhum problema grave de encoding detectado.


##  Architectural Decisions – Go/No-Go para Bronze

1. **Schema manual será adotado oficialmente**  
   - StructType definido antes da ingestão Bronze.
   - Tipos revisados com base no profiling.

2. **Todos os CSVs serão ingeridos em Delta Lake (Bronze layer)**  
   - Fidelidade ao raw (raw-but-clean).
   - Enriquecimento apenas com ingestion metadata.

3. **Data cleansing mínimo na Bronze**  
   - Filtrar arquivos inválidos.
   - Normalizar header/colunas se necessário.

4. **Silver receberá dados normalizados**  
   - Chave técnica: `artist_name_key = lower(trim(artist_name))`
   - Atributo de exibição: `artist_name_display`
   - Remoção de MBID da dimensão (mantido só no Bronze).

5. **playcount será convertido para LongType**  
   - Previne erros futuros em agregações.

6. **Partition Strategy**  
   - Como os datasets não possuem partição temporal nativa,
     Bronze será salva sem particionamento explícito.

7. **Observability**  
   - Criar contadores de volume por ingestão.
   - Registrar ingestion_date + source_file.



##  Próximos Passos – Bronze Ingestion Pipeline

### Step 1 — Definir o Schema Manual
- Criar StructType para cada dataset:
  - top_artists
  - top_albums
  - top_tracks

### Step 2 — Criar o Notebook Bronze
- Leitura com `.schema()`
- Exclusão de arquivos “._*”
- Conversão de tipos (playcount → LongType)
- Inclusão de colunas técnicas:
  - ingestion_date
  - source_file

### Step 3 — Persistência Bronze
- Salvar com `.write.format("delta")`
- Criar estrutura de diretórios:
  - bronze/top_artists
  - bronze/top_albums
  - bronze/top_tracks

### Step 4 — Validação Pós-Ingestão
- count()
- schema()
- comparação com profiling

### Step 5 — Preparação para Silver
- notebook da dimensão artist
- notebook da dimensão album
- notebook da dimensão track
- notebook de fato(s)
